In [9]:
import cv2
import numpy as np
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from torchvision.models import VGG

In [28]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = np.array([cv2.resize(im, (128, 128)) for im in X_train])
X_test = np.array([cv2.resize(im, (128, 128)) for im in X_test])

In [33]:
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

In [11]:
X_train.shape

(50000, 128, 128, 3)

In [36]:
from keras.applications import VGG16
from keras import Model
from keras.layers import Dense, Flatten

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Zamień warstwy w pełni połączone na własne
x = base_model.output
x = Flatten()(x)  # Dodaj warstwę Flatten, aby przekształcić dane z warstw konwolucyjnych do jednego wymiaru
x = Dense(128, activation='relu')(x)
x = Dense(10, activation='softmax')(x)

# Utwórz nowy model
model = Model(inputs=base_model.input, outputs=x)

for layer in base_model.layers:
    layer.trainable = False

In [37]:
from keras.optimizers import Adam

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [38]:
from keras_preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)

In [39]:
batch_size = 32
epochs = 10

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/10
1563/1563 [==============================] - 1909s 1s/step - loss: 0.9070 - accuracy: 0.7387
Epoch 2/10
1563/1563 [==============================] - 1854s 1s/step - loss: 0.4725 - accuracy: 0.8404
Epoch 3/10
1563/1563 [==============================] - 1837s 1s/step - loss: 0.3711 - accuracy: 0.8732
Epoch 4/10
1563/1563 [==============================] - 1849s 1s/step - loss: 0.3126 - accuracy: 0.8915
Epoch 5/10
1563/1563 [==============================] - 1890s 1s/step - loss: 0.2615 - accuracy: 0.9112
Epoch 6/10
1563/1563 [==============================] - 1945s 1s/step - loss: 0.2211 - accuracy: 0.9242
Epoch 7/10
1563/1563 [==============================] - 1980s 1s/step - loss: 0.1917 - accuracy: 0.9346
Epoch 8/10
1563/1563 [==============================] - 1903s 1s/step - loss: 0.1660 - accuracy: 0.9443
Epoch 9/10
1563/1563 [==============================] - 1865s 1s/step - loss: 0.1443 - accuracy: 0.9505
Epoch 10/10
1563/1563 [==============================] - 1874s 1

In [34]:
y_train.shape

(50000, 10)

In [40]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

313/313 - 373s - loss: 1.1752 - accuracy: 0.7964 - 373s/epoch - 1s/step

Test accuracy: 0.7964000105857849


In [41]:
model.save('NN/cifar10_vgg.h5')